# General Assembly DSI - Denver 2018
## Capstone Project - DFS Model
This is my capstone project at General Assembly's fifth [Data Science Immersive](https://generalassemb.ly/education/data-science-immersive) cohort in 2018. I am developing a model to assist in optimizing NFL lineups on the daily fantasy sports platforms [Draft Kings](https://www.draftkings.com/) and [Fan Duel](https://www.fanduel.com/).

### Problem Statement

Can we build a model to predict a football player’s fantasy football performance to estimate their value and implement the model in conjunction with a daily fantasy strategy to be profitable?

In [1]:
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.decomposition import PCA
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical

%matplotlib inline

/Users/charleydixon/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
/Users/charleydixon/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Couldn't import dot_parser, loading of dot files will not be possible.


In [2]:
# load data
data = pd.read_csv('../data/modeling_rbs.csv')

In [3]:
data = data.groupby(['Name', 'Year', 'Week', 'Month', 'Team', 'Oppt']).sum()

In [4]:
data.head()

Age  h/a  Favored  Spread  \
Name            Year Week Month     Team Oppt                                 
Abdullah, Ameer 2015 2    September DET  MIN   22.099    0        0    -2.5   
                     3    September DET  DEN   22.106    1        0    -3.0   
                     4    October   DET  SEA   22.114    0        0   -10.0   
                     5    October   DET  ARI   22.120    1        0    -4.5   
                     6    October   DET  CHI   22.127    1        1    -3.5   

                                                O/U  Temperature  Wind  \
Name            Year Week Month     Team Oppt                            
Abdullah, Ameer 2015 2    September DET  MIN   44.0         70.0  12.0   
                     3    September DET  DEN   45.0         72.0   0.0   
                     4    October   DET  SEA   43.0         61.0   2.0   
                     5    October   DET  ARI   46.0         72.0   0.0   
                     6    October   DET  CHI   45.5         72.0   0.0   

                                                Carries   Rush_Yds  Rush_TDs  \
Name            Year Week Month     Team Oppt                                  
Abdullah, Ameer 2015 2    September DET  MIN   8.000000  46.333333  0.333333   
                     3    September DET  DEN   7.666667  41.666667  0.333333   
                     4    October   DET  SEA   7.333333  23.666667  0.000000   
                     5    October   DET  ARI   9.000000  21.666667  0.000000   
                     6    October   DET  CHI   9.000000  24.000000  0.000000   

                                                 ...      Weather_DOME  \
Name            Year Week Month     Team Oppt    ...                     
Abdullah, Ameer 2015 2    September DET  MIN     ...                 0   
                     3    September DET  DEN     ...                 1   
                     4    October   DET  SEA     ...                 0   
                     5    October   DET  ARI     ...                 1   
                     6    October   DET  CHI     ...                 1   

                                               Weather_Fog  Weather_Rain  \
Name            Year Week Month     Team Oppt                              
Abdullah, Ameer 2015 2    September DET  MIN             0             0   
                     3    September DET  DEN             0             0   
                     4    October   DET  SEA             0             0   
                     5    October   DET  ARI             0             0   
                     6    October   DET  CHI             0             0   

                                               Weather_Rain | Fog  \
Name            Year Week Month     Team Oppt                       
Abdullah, Ameer 2015 2    September DET  MIN                    0   
                     3    September DET  DEN                    0   
                     4    October   DET  SEA                    0   
                     5    October   DET  ARI                    0   
                     6    October   DET  CHI                    0   

                                               Weather_Snow  \
Name            Year Week Month     Team Oppt                 
Abdullah, Ameer 2015 2    September DET  MIN              0   
                     3    September DET  DEN              0   
                     4    October   DET  SEA              0   
                     5    October   DET  ARI              0   
                     6    October   DET  CHI              0   

                                               Weather_Snow | Fog  \
Name            Year Week Month     Team Oppt                       
Abdullah, Ameer 2015 2    September DET  MIN                    0   
                     3    September DET  DEN                    0   
                     4    October   DET  SEA                    0   
                     5    October   DET  ARI                    0   
                     6  

### Modeling
- Linear Regression
- Random Forest
- Support Vector Regression
- Boosting
- PCA
- Neural Net

### Linear Regression

In [5]:
columns = [col for col in data.columns if col != 'FD points']

def linreg(df, features, target = 'FD points'):
    X = df[features]
    y = df[target]

    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

    # first attempt - all features
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    print('=============EVAULATION=============\n')
    print('R2 Training:', model.score(X_train, y_train))
    print('R2 Testing:', model.score(X_test, y_test))

    predictions = model.predict(X_test)

    print('RMSE:', np.sqrt(mean_squared_error(y_test, predictions)))
    print('MAE:', mean_absolute_error(y_test, predictions), '\n')
    print('=============COEFFICIENTS=============\n')
    print('Intercept:', model.intercept_)
    for key, index in dict(zip(X.columns, model.coef_)).items():
        print(key, index)
    return model

lr_all_features = linreg(data, columns)

=============EVAULATION=============

R2 Training: 0.29202997295009725
R2 Testing: 0.31895794910927167
RMSE: 6.273472890532594
MAE: 4.817716067101606 

=============COEFFICIENTS=============

Intercept: -7.9660028953115045
Age 0.012090830463540385
h/a 0.7056492165069178
Favored 0.49918014941457517
Spread 0.004801744493945273
O/U 0.0787057374499108
Temperature -0.029221087281389172
Wind -0.06247126717026167
Carries 0.14147398113490475
Rush_Yds 0.024128986258513948
Rush_TDs 0.22323631808922761
Receptions -0.056855022932739384
Targets 0.559679892919739
Rec_Yds 0.011068438057728183
Rec_TDs -0.3002818069614838
Opp_Avg_Carries 0.09502522706385733
Opp_Rank 0.05295639825473429
Opp_Avg_TDs_Allowed 0.2892627781673079
Opp_Avg_Yds_Allowed -0.004368493006366178
Weather_DOME -0.6750207928236073
Weather_Fog -2.4719962221717267
Weather_Rain -0.15603564562047867
Weather_Rain | Fog 1.1977752815290368
Weather_Snow 1.0690802909972468
Weather_Snow | Fog 1.6947501973894517
Weather_Snow | Freezing Rain 0.0
W

Feature Coefficients that seem logical:
- Increasing Prediction for:
    - Home/Away
    - Favored
    - O/U
    - Attempts
    - Completions
    - TDs
    - Y/A
    
- Decreasing Prediction for:
    - Age
        - Sort of...
    - Wind
    - Rain
    - Rain & Fog
    - Snow
    - Snow & Fog
    - Interceptions
    - Opponent Rank
        - The rank columns is actually backwards so defenses get better as ranking improves

Feature Coefficients that make no sense to me:
- Increasing Prediction for:
    - Fog
    - Snow & Freezing Rain
    - Opponent Interceptions
- Decreasing Prediction for:
    - Being in a Dome (Controlled conditions seem like a player should perform better)
    - Sunny weather
    - Yards
    - Opponent Attempts Allowed
    - Opponent Completions Allowed
    - Opponent TDs Allowed
- Almost no effect from:
    - Rating
    - FD Salary
        - Considering salary kind of derives FanDuel's predictions I feel like this would have an impact.

**The most interesting thing to note here is that FD Salary has seemingly no effect on a player's point production**

In [6]:
# data.columns

In [7]:
# remove_cols = ['Weather_Fog', 'Weather_Snow | Freezing Rain', 'Opp_Avg_Ints', 'Weather_DOME', 
#                'Weather_Sunny', 'Yards', 'Opp_Avg_Att_Allowed', 'Opp_Avg_Comp_Allowed', 
#                'Opp_Avg_TDs_Allowed', 'Rating', 'FD salary', 'FD points']

# features = [col for col in data.columns if col not in remove_cols]

In [8]:
# lr_partial_features = linreg(data, features)

### Random Forest Regression

In [9]:
features = [col for col in data.columns if col != 'FD points']

X = data[features]
y = data['FD points']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

#### No Hypertuning

In [10]:
model = RandomForestRegressor()
cross_val_score(model, X_train, y_train).mean()

0.17327925181727474

In [11]:
model.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [12]:
model.score(X_test, y_test)

0.23823573099659

In [13]:
predictions = model.predict(X_test)

In [14]:
mean_absolute_error(y_test, predictions)

5.06633909287257

In [15]:
np.sqrt(mean_squared_error(y_test, predictions))

6.63485384481954

#### With Hypertuning

In [16]:
model = RandomForestRegressor()

params = {
    'max_depth': [10, 20, 30],
    'n_estimators': [70, 75, 80],
    'min_samples_leaf': [13, 14],
    'max_features': [10, 20]
}

gs = GridSearchCV(model, param_grid = params)
gs.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'max_depth': [10, 20, 30], 'n_estimators': [70, 75, 80], 'min_samples_leaf': [13, 14], 'max_features': [10, 20]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [17]:
print(gs.best_score_)
print(gs.score(X_test, y_test))
print(gs.best_params_)

0.2818799772319774
0.32867750331679496
{'max_depth': 10, 'max_features': 10, 'min_samples_leaf': 14, 'n_estimators': 75}


In [18]:
y_pred = gs.best_estimator_.predict(X_test)
print(mean_absolute_error(y_test, y_pred))
print(np.sqrt(mean_squared_error(y_test, y_pred)))

4.7630812920862
6.228545796810142


In [19]:
dict(zip(X.columns, gs.best_estimator_.feature_importances_))

{'Age': 0.0375676968294432,
 'h/a': 0.007698234704843623,
 'Favored': 0.0054764950424066264,
 'Spread': 0.016573108494078843,
 'O/U': 0.02289060370403699,
 'Temperature': 0.026598648014164377,
 'Wind': 0.013881196405668372,
 'Carries': 0.15279007919864043,
 'Rush_Yds': 0.11750768662648566,
 'Rush_TDs': 0.021144465958137508,
 'Receptions': 0.054988165051562315,
 'Targets': 0.056143301389956914,
 'Rec_Yds': 0.0432257192700972,
 'Rec_TDs': 0.002185939767923902,
 'Opp_Avg_Carries': 0.025285124432440527,
 'Opp_Rank': 0.030298818834626182,
 'Opp_Avg_TDs_Allowed': 0.010085836448933388,
 'Opp_Avg_Yds_Allowed': 0.032377137971133264,
 'Weather_DOME': 0.0014678565553502287,
 'Weather_Fog': 0.0,
 'Weather_Rain': 0.0,
 'Weather_Rain | Fog': 0.0,
 'Weather_Snow': 0.0,
 'Weather_Snow | Fog': 0.0,
 'Weather_Snow | Freezing Rain': 0.0,
 'Weather_Sunny': 0.0035711220299021154,
 'FD salary': 0.3182427632701683}

> Now this is telling me that `FD Salary` is far and away the most important feature

### Support Vector Regression

In [20]:
features = [col for col in data.columns if col != 'FD points']

X = data[features]
y = data['FD points']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

#### No Hypertuning

In [21]:
model = SVR()

In [22]:
model.fit(X_train, y_train)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [23]:
model.score(X_train, y_train)

0.11852806948272077

In [24]:
model.score(X_test, y_test)

-0.053384621940437915

In [25]:
y_pred = model.predict(X_test)

In [26]:
mean_absolute_error(y_test, y_pred)

5.880975597911216

#### Hypertuning

In [63]:
model = SVR()
params = {
    'kernel': ['linear'],
    'C': [0.1, 0.01]
}

gs = GridSearchCV(model, param_grid = params)
gs.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'kernel': ['linear'], 'C': [0.1, 0.01]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [64]:
print(gs.best_score_)
print(gs.score(X_test, y_test))
print(gs.best_params_)

0.2570700423863125
0.31175043172921624
{'C': 0.1, 'kernel': 'linear'}


In [65]:
y_pred = gs.best_estimator_.predict(X_test)
print(mean_absolute_error(y_test, y_pred))
print(np.sqrt(mean_squared_error(y_test, y_pred)))

4.721089344752972
6.306581831620253


### ADA Boost

In [30]:
features = [col for col in data.columns if col != 'FD points']

X = data[features]
y = data['FD points']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [31]:
model = AdaBoostRegressor()
params = {
    'n_estimators': [45, 47, 50],
    'learning_rate': [.8, .83, .85, .9]
}

gs = GridSearchCV(model, param_grid=params)
gs.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=AdaBoostRegressor(base_estimator=None, learning_rate=1.0, loss='linear',
         n_estimators=50, random_state=None),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_estimators': [45, 47, 50], 'learning_rate': [0.8, 0.83, 0.85, 0.9]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [32]:
print(gs.best_score_)
print(gs.score(X_test, y_test))
print(gs.best_params_)

0.09703077506486643
0.07717861014595317
{'learning_rate': 0.85, 'n_estimators': 47}


In [33]:
y_pred = gs.best_estimator_.predict(X_test)
print(mean_absolute_error(y_test, y_pred))
print(np.sqrt(mean_squared_error(y_test, y_pred)))

6.203322501963749
7.302639933556975


### Gradient Boost

In [34]:
features = [col for col in data.columns if col != 'FD points']

X = data[features]
y = data['FD points']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [35]:
model = GradientBoostingRegressor()
params = {
    'n_estimators': [75, 80, 85],
    'learning_rate': [0.15, 0.2, 0.25, 0.3]
}

gs = GridSearchCV(model, param_grid=params)
gs.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_estimators': [75, 80, 85], 'learning_rate': [0.15, 0.2, 0.25, 0.3]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [36]:
print(gs.best_score_)
print(gs.score(X_test, y_test))
print(gs.best_params_)

0.2461230603205772
0.32104313889223357
{'learning_rate': 0.15, 'n_estimators': 75}


In [37]:
y_pred = gs.best_estimator_.predict(X_test)
print(mean_absolute_error(y_test, y_pred))
print(np.sqrt(mean_squared_error(y_test, y_pred)))

4.785058979735102
6.263861582434477


### Principal Component Analysis

In [54]:
features = [col for col in data.columns if col != 'FD points']

X = data[features]
y = data['FD points']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [55]:
ss = StandardScaler()
X_train_scaled = ss.fit_transform(X_train)
X_test_scaled = ss.transform(X_test)

pca = PCA(n_components = 20)
pca.fit(X_train_scaled)

PCA(copy=True, iterated_power='auto', n_components=20, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [56]:
var_exp = pca.explained_variance_ratio_
print('Explained Variance: ', var_exp)

Explained Variance:  [0.14314189 0.0961016  0.09242599 0.07789782 0.05272022 0.0512491
 0.04190068 0.04081237 0.0400064  0.03905669 0.03850776 0.03751357
 0.03603068 0.0315258  0.03060055 0.02727189 0.02590727 0.02492588
 0.02146407 0.01808583]


In [57]:
cum_var_exp = np.cumsum(var_exp)
print('Cumulative Explained Variance: ', cum_var_exp)

Cumulative Explained Variance:  [0.14314189 0.23924349 0.33166948 0.4095673  0.46228752 0.51353663
 0.55543731 0.59624968 0.63625608 0.67531276 0.71382053 0.7513341
 0.78736478 0.81889058 0.84949113 0.87676302 0.90267028 0.92759616
 0.94906023 0.96714606]


In [58]:
Z_train = pca.transform(X_train_scaled)
Z_test = pca.transform(X_test_scaled)

In [59]:
model = LinearRegression()

model.fit(Z_train, y_train)
print(model.score(Z_train, y_train))
print(model.score(Z_test, y_test))

0.28480683136238427
0.31458480714383197


In [60]:
predictions = model.predict(Z_test)

In [61]:
mean_absolute_error(y_test, predictions)

4.815208667886761

In [62]:
np.sqrt(mean_squared_error(y_test, predictions))

6.293582432191537

### Neural Network

In [47]:
features = [col for col in data.columns if col != 'FD points']

X = data[features]
y = data['FD points']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [48]:
ss = StandardScaler()
X_train_scaled = ss.fit_transform(X_train)
X_test_scaled = ss.transform(X_test)

In [49]:
model = Sequential()

model.add(Dense(29, activation = 'relu', input_dim = X_train.shape[1]))
model.add(Dense(5, activation = 'relu'))
model.add(Dense(1, activation=None))

In [50]:
model.compile(loss = 'mean_squared_error', optimizer = 'adam')

In [51]:
model.fit(X_train_scaled, 
          y_train, 
          epochs = 10, 
          batch_size = 2, 
          validation_data = (X_test_scaled, y_test))

Train on 2777 samples, validate on 926 samples
Epoch 1/10
2777/2777 [==============================] - 1s 538us/step - loss: 57.6637 - val_loss: 41.1680
Epoch 2/10
2777/2777 [==============================] - 1s 459us/step - loss: 44.0415 - val_loss: 40.5211
Epoch 3/10
2777/2777 [==============================] - 1s 484us/step - loss: 42.6784 - val_loss: 40.2471
Epoch 4/10
2777/2777 [==============================] - 1s 447us/step - loss: 41.9212 - val_loss: 40.5788
Epoch 5/10
2777/2777 [==============================] - 1s 440us/step - loss: 41.4828 - val_loss: 40.0503
Epoch 6/10
2777/2777 [==============================] - 1s 510us/step - loss: 40.8900 - val_loss: 40.0830
Epoch 7/10
2777/2777 [==============================] - 1s 532us/step - loss: 40.4700 - val_loss: 40.5181
Epoch 8/10
2777/2777 [==============================] - 2s 559us/step - loss: 39.6147 - val_loss: 41.6096
Epoch 9/10
2777/2777 [==============================] - 1s 530us/step - loss: 39.5736 - val_loss: 40.5488

In [52]:
predictions = model.predict(X_test_scaled)

In [53]:
print(mean_absolute_error(y_test, predictions))
print(np.sqrt(mean_squared_error(y_test, predictions)))

4.8013910098415735
6.361829399117297
